In [11]:
import pandas as pd
import os
import json

In [12]:
def fetch_results(dataset_name, model_name, dataset_split):
    results = []
    directory = f"generated_quiz_answers/{dataset_name}/{model_name}/{dataset_split}"
    model_name = model_name.split("_")[0] if "rerun" in model_name else model_name

    for language in os.listdir(directory):
        language_directory = os.path.join(directory, language)
        json_file = os.path.join(language_directory, "contamination.json")
        df_file = os.path.join(language_directory, "quiz_answers.csv")
        df_ans = pd.read_csv(df_file)

        with open(json_file) as f:
            data = json.load(f)
            score = data["score"]
            contamination = data["contamination"]
            results.append(
                {
                    "Language": language,
                    f"Score": score,
                    f"Contamination": contamination,
                    f"Total_points": len(df_ans),
                    f"Model": model_name,
                    f'Split': dataset_split,
                    f"Dataset": dataset_name,
                }
            )

    df = pd.DataFrame(results)
    return df

In [13]:
# list all directories in generated_quiz_answers
dataset_names = os.listdir("generated_quiz_answers")
models = ["dev-moonshot","palm-32k"]
# results = {}
df = pd.DataFrame()
for dataset_name in dataset_names:
    for model_name in models:
        # fetch results for each dataset from generated_quiz_answers -> dataset_name -> model_name
        # if model_name in generated_quiz_answers/dataset_name
        if os.path.exists(f"generated_quiz_answers/{dataset_name}/{model_name}_rerun"):
            model_name = f"{model_name}_rerun"
        if os.path.exists(f"generated_quiz_answers/{dataset_name}/{model_name}/test"):
            split = "test"
        else:
            split = "validation"
        try:   
            df = pd.concat([df, fetch_results(dataset_name, model_name, split)])
        except:
            print(f"Error in {dataset_name} {model_name}")
            continue

Error in xcopa dev-moonshot
Error in tydiqa dev-moonshot


In [14]:
results_udpos_gpt = fetch_results("udpos", "dev-moonshot", "test")

In [15]:
results_udpos_palm = fetch_results("udpos", "palm-32k", "test")

In [16]:
results_udpos_palm

,Language,Score_palm-32k,Contamination_palm-32k,Total_points_palm-32k
0,fr,0.34,0.120000,109
1,zh,0.15,-0.133333,110
2,el,0.16,-0.120000,108
3,bg,0.10,-0.200000,110
4,fi,0.15,-0.133333,110
5,pl,0.09,-0.213333,110
6,it,0.23,-0.026667,106
7,et,0.10,-0.200000,109
8,ro,0.15,-0.133333,109
9,hu,0.13,-0.160000,106


In [14]:
df

,Language,Score,Contamination,Total_points,Model,Split,Dataset
0,es,0.743902,0.658537,82,dev-moonshot,test,paws-x
1,fr,0.781250,0.708333,96,dev-moonshot,test,paws-x
2,zh,0.735849,0.647799,53,dev-moonshot,test,paws-x
3,de,0.829787,0.773050,94,dev-moonshot,test,paws-x
4,ko,0.583333,0.444444,60,dev-moonshot,test,paws-x
...,...,...,...,...,...,...,...
4,ja,0.790000,0.720000,141,palm-32k,validation,tydiqa
5,id,0.370000,0.160000,129,palm-32k,validation,tydiqa
6,th,0.380000,0.173333,144,palm-32k,validation,tydiqa
7,en,0.730000,0.640000,139,palm-32k,validation,tydiqa


In [17]:

# join the two dataframes on language
results_udpos_combined = results_udpos_gpt.merge(results_udpos_palm, on="Language")

In [18]:
results_udpos_combined

,Language,Score_dev-moonshot,Contamination_dev-moonshot,Total_points_dev-moonshot,Score_palm-32k,Contamination_palm-32k,Total_points_palm-32k
0,fr,0.32,0.093333,110,0.34,0.120000,109
1,zh,0.30,0.066667,110,0.15,-0.133333,110
2,el,0.38,0.173333,110,0.16,-0.120000,108
3,bg,0.29,0.053333,110,0.10,-0.200000,110
4,fi,0.28,0.040000,110,0.15,-0.133333,110
5,pl,0.24,-0.013333,110,0.09,-0.213333,110
6,it,0.27,0.026667,109,0.23,-0.026667,106
7,et,0.37,0.160000,109,0.10,-0.200000,109
8,ro,0.06,-0.253333,110,0.15,-0.133333,109
9,hu,0.40,0.200000,110,0.13,-0.160000,106


In [ ]:
results_